In [5]:
def get_error_ulrssm(log_file, categories):
    
    # log_file = '/home/s94zalek_hpc/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class/log/test_dt4d_intra_class_20241029_122533.log'

    with open(log_file, 'r') as f:
        lines = f.readlines()
        
    # print('lines', len(lines))
    
    # filter all lines that have 'Val geo error: ' in them
    err_by_category = {category: 0 for category in categories}
    times_by_category = {category: 0 for category in categories}

    for line in lines:
        if 'Val geo error: ' in line:
            
            # each line has form 2024-10-29 16:54:20,347 INFO: Val geo error: data/DT4D_r/off/zlorp/DancingRunningMan325.off-data/DT4D_r/off/zlorp/DancingRunningMan272.off, 1.0841
            # get two file names and the error
            
            file_names = line.split('Val geo error: ')[1].split(', ')[0]
            file_name_1, file_name_2 = file_names.split('-')
            
            category_1 = file_name_1.split('/')[-2]
            category_2 = file_name_2.split('/')[-2]
            
            error = float(line.split('Val geo error: ')[1].split(', ')[1])
            
            # print(file_names)
            # print(category_1, category_2, error)
            
            err_by_category[category_1] += error
            err_by_category[category_2] += error
            
            times_by_category[category_1] += 1
            times_by_category[category_2] += 1
            
    for category in categories:
        err_by_category[category] /= times_by_category[category] if times_by_category[category] > 0 else 1
        
    return err_by_category, times_by_category


In [6]:
import my_code.diffusion_training_sign_corr.data_loading as data_loading

models_logs = {
    'inter': {
        '0': [
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_inter_class_faust_0_archived_20250228_085707/log/test_dt4d_inter_class_faust_0_20250228_022757.log',
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_inter_class_faust_0_archived_20250228_022757/log/test_dt4d_inter_class_faust_0_20250227_195759.log',
        ],
        '5': [
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_inter_class_faust_5_archived_20250228_032533/log/test_dt4d_inter_class_faust_5_20250227_195759.log',
        ],
        '10': [
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_inter_class_faust_10_archived_20250228_042255/log/test_dt4d_inter_class_faust_10_20250227_195800.log',
        ],
        '15': [
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_inter_class_faust_15_archived_20250228_051259/log/test_dt4d_inter_class_faust_15_20250227_195800.log',
        ],
    },
    'intra': {
        '0': [
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class_faust_0_archived_20250228_045557/log/test_dt4d_intra_class_faust_0_20250228_002817.log',
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class_faust_0_archived_20250228_002817/log/test_dt4d_intra_class_faust_0_20250227_195759.log',
        ],
        '5': [
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class_faust_5_archived_20250228_061114/log/test_dt4d_intra_class_faust_5_20250228_010508.log',
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class_faust_5_archived_20250228_010508/log/test_dt4d_intra_class_faust_5_20250227_195759.log',
        ],
        '10': [
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class_faust_10_archived_20250228_072853/log/test_dt4d_intra_class_faust_10_20250228_014302.log',
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class_faust_10_archived_20250228_014302/log/test_dt4d_intra_class_faust_10_20250227_195800.log',
        ],
        '15': [
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class_faust_15_archived_20250228_085157/log/test_dt4d_intra_class_faust_15_20250228_022555.log',
            '/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class_faust_15_archived_20250228_022555/log/test_dt4d_intra_class_faust_15_20250227_195800.log',
        ],
    },
}

for split_type in ['inter', 'intra']:
    
    print(f'---{split_type}')
    
    for model_type in ['0', '5', '10', '15']:
        
        dataset_category = f'DT4D_{split_type}_pair'
        single_dataset, test_dataset = data_loading.get_val_dataset(
            dataset_category,
            'test', 200, preload=False, return_evecs=True, centering='bbox'
            )

        categories = []

        for file in single_dataset.off_files:
            categories.append(file.split('/')[-2])
            
        categories = list(set(categories))
        
        err_by_category_ulrssm_total = {category: 0 for category in categories}
        err_by_category_ulrssm_list = {category: [] for category in categories}

        for ulrssm_log_file in models_logs[split_type][model_type]:
            err_by_category_ulrssm, times_by_category_ulrssm = get_error_ulrssm(
                ulrssm_log_file,
                categories,
            )
            
            for category in categories:
                err_by_category_ulrssm_total[category] += err_by_category_ulrssm[category]
                err_by_category_ulrssm_list[category].append(err_by_category_ulrssm[category])
        
        for category in categories:
            err_by_category_ulrssm_total[category] /= len(models_logs[split_type][model_type])

        # sort categories by error
        categories_sorted = sorted(categories, key=lambda x: err_by_category_ulrssm_total[x])

        print(f'# Refinement {model_type}')

        for category in categories_sorted:
            print(f'{category}: ulrssm : {err_by_category_ulrssm_total[category]:.1f}', end=' ')
            
            # print err_by_category_ulrssm_list[category], but round all numbers to 1 decimal place
            print('[' + ', '.join([f'{x:.1f}' for x in err_by_category_ulrssm_list[category]]) + ']')
            
        print()
 

In [1]:
import numpy as np
import my_code.diffusion_training_sign_corr.data_loading as data_loading
import yaml
from tqdm import tqdm
import metrics.geodist_metric as geodist_metric
from utils.shape_util import compute_geodesic_distmat
import torch


dataset_name = 'DT4D_intra_pair'

single_dataset, pair_dataset = data_loading.get_val_dataset(
    dataset_name, 'test', 128, preload=False, return_evecs=True, centering='bbox'
)


In [2]:
dist_mat_list = []

for i in tqdm(range(len(single_dataset))):
# for i in tqdm(range(2)):

    data_i = single_dataset[i]

    dist_mat = torch.tensor(
        compute_geodesic_distmat(data_i['verts'].numpy(), data_i['faces'].numpy())    
    )
    
    dist_mat_list.append(dist_mat)

In [1]:
len(dist_mat_list)

In [3]:
categories = []

for file in single_dataset.off_files:
    categories.append(file.split('/')[-2])
    
categories = list(set(categories))
print(categories)

In [4]:

path = f'/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class_faust_15_archived_20250228_085157/visualization'

err_by_category = {category: 0 for category in categories}
times_by_category = {category: 0 for category in categories}

geo_err_list = []

for i in tqdm(range(len(pair_dataset))):
# for i in tqdm(range(2)):
    
    data_i = pair_dataset[i]
    
    first_idx = data_i['first']['id']
    second_idx = data_i['second']['id']
    
    # print(first_idx, second_idx)
    
    off_first = single_dataset.off_files[first_idx]
    name_first = off_first.split('/')[-1].split('.')[0]
    
    off_second = single_dataset.off_files[second_idx]
    name_second = off_second.split('/')[-1].split('.')[0]
    
    # print(name_first, name_second)
    
    # break    
    
    
    # p2p = torch.tensor(
    #     np.loadtxt(f'{path}/{name_first}_{name_second}.txt')
    # ).int()
    
    p2p = torch.tensor(
        torch.load(f'{path}/{i}.pth')
    ).int()
    
    dist_x = dist_mat_list[first_idx]
    dist_y = dist_mat_list[second_idx]
    
    # print(i, p2p.shape, dist_x.shape, dist_y.shape)
    
    corr_first = data_i['first']['corr']
    corr_second = data_i['second']['corr']
    
    geo_err = geodist_metric.calculate_geodesic_error(
        dist_x, corr_first.cpu(), corr_second.cpu(), p2p, return_mean=True
    ) * 100
    geo_err_list.append(geo_err)

    # print(geo_err)

    category_first = (single_dataset.off_files[int(first_idx)]).split('/')[-2]
    category_second = (single_dataset.off_files[int(second_idx)]).split('/')[-2]
            
            # if 'hippo' in category_first or 'hippo' in category_second:
            #     continue
            
    err_by_category[category_first] += float(geo_err)
    err_by_category[category_second] += float(geo_err)
    
    times_by_category[category_first] += 1
    times_by_category[category_second] += 1
            

    
geo_err_list = torch.tensor(geo_err_list)
print(f'{dataset_name}, mean geo err: {geo_err_list.mean()}, median: {geo_err_list.median()}')

for category in categories:
    err_by_category[category] /= times_by_category[category] if times_by_category[category] > 0 else 1
       

In [6]:
err_by_category_list = sorted(
    [(k, v) for k, v in err_by_category.items()],
    key=lambda x: x[1], reverse=False
)
categories_sorted = [entry[0] for entry in err_by_category_list]

for category in categories_sorted:
    print(f'{category}: {err_by_category[category]:.1f} ({times_by_category[category]})')

In [12]:
torch.load('/home/s94zalek_hpc/baselines/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/results/dt4d_intra_class_faust_15_archived_20250228_085157/visualization/99.pth').shape